In [1]:
import pandas as pd

In [2]:
train_rides_df = pd.read_csv('DBtrainrides.csv')

In [3]:
train_rides_df.head()

,ID,line,path,eva_nr,category,station,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check
0,1573967790757085557-2407072312-14,20,Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time
1,349781417030375472-2407080017-1,18,NaN,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,NaN,2024-07-08 00:17:00,NaN,NaN,0,0,NaN,on_time,on_time
2,7157250219775883918-2407072120-25,1,Hamm(Westf)Hbf|Kamen|Kamen-Methler|Dortmund-Ku...,8000406,4,Aachen-Rothe Erde,Nordrhein-Westfalen,Aachen,52066,6.116475,50.770202,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,0,0,NaN,on_time,on_time
3,349781417030375472-2407080017-2,18,Aachen Hbf,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,NaN,NaN,0,0,NaN,on_time,on_time
4,1983158592123451570-2407080010-3,33,Herzogenrath|Kohlscheid,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,2024-07-08 00:20:00,2024-07-08 00:21:00,0,0,NaN,on_time,on_time


In [4]:
zip_df=train_rides_df.groupby("zip").size().reset_index(name="count")
zip_df.head()

,zip,count
0,1067,2458
1,1069,2045
2,1097,3305
3,1109,1800
4,1127,597


In [5]:
train_rides_df.count()

ID                       2061357
line                     2061357
path                     1850002
eva_nr                   2061357
category                 2061357
station                  2061357
state                    2061357
city                     2061357
zip                      2061357
long                     2061357
lat                      2061357
arrival_plan             1850002
departure_plan           2061357
arrival_change           1585727
departure_change         1721431
arrival_delay_m          2061357
departure_delay_m        2061357
info                      645341
arrival_delay_check      2061357
departure_delay_check    2061357
dtype: int64

2,061,357–1,850,002 = 211,355 fewer cases in the 'path' column

In [6]:
# Step 2: Create a new DataFrame with a count of how often each 'eva_nr' is mentioned
station_mapping_df = train_rides_df.groupby(['eva_nr', 'station', 'category', 'city', 'long', 'lat']).size().reset_index(name='count')

# Step 3: Optionally, reset the index for a cleaner DataFrame
station_mapping_df = station_mapping_df.reset_index(drop=True)

# Display the new DataFrame
station_mapping_df.head()

,eva_nr,station,category,city,long,lat,count
0,8000001,Aachen Hbf,2,Aachen,6.091499,50.767800,1488
1,8000002,Aalen Hbf,3,Aalen,10.096271,48.841013,823
2,8000004,Altenbeken,3,Altenbeken,8.943319,51.766433,848
3,8000007,Alzey,3,Alzey,8.109749,49.750200,591
4,8000009,Ansbach,3,Ansbach,10.578239,49.298032,829


In [7]:
station_mapping_df.count()

eva_nr      1996
station     1996
category    1996
city        1996
long        1996
lat         1996
count       1996
dtype: int64

what is the average number of stations per category? go check the 'category' column

In [8]:
# # Step 1: Split the 'path' into individual stations where 'path' is not empty
# # Create a new DataFrame that splits the 'path' column
# stations_df = train_rides_df['path'].str.split('|', expand=True)
# 
# # Step 2: Keep the same 'eva_nr', 'station', and 'city' columns, add the split station columns
# split_stations_df = pd.concat([train_rides_df[['ID','eva_nr', 'station', 'city']], stations_df], axis=1)
# 
# split_stations_df.head()

In [ ]:
# Step 1: Split the 'path' into individual stations where 'path' is not empty
train_rides_df['path'] = train_rides_df['path'].fillna('')

# Step 2: Split each 'path' entry by '|', creating multiple rows per station
split_stations_df = train_rides_df.assign(station_name=train_rides_df['path'].str.split('|')).explode('station_name').drop(columns=['path'])

# Display the result
split_stations_df.head()

In [9]:
# split_stations_df.head()

Visualise the stations in a map (/grid) (without it being mapped in a real map) and see if there are any patterns. George \\

Additionally, use the same map to see the delay (in % of total rides) per station and per zip. Delay in this case would be if the train did not arrive / leave at the intended time. George \\


See if we can use K-means to cluster the stations into mini regions to see if the entire region has delay problems. We do not know what k to use, so we can try a few values. Check the distance per stations and see by eye if the clusters make sense to create. Raphael

In [10]:
# import dask.dataframe as dd

In [11]:
# # Load your DataFrame as a Dask DataFrame
# dask_df = dd.from_pandas(train_rides_df, npartitions=10)
# 
# # Step 1: Find the maximum number of splits by checking the station counts in each 'path'
# max_stations = dask_df['path'].str.count('|').max().compute() + 1
# 
# # Step 2: Split the 'path' column into the calculated number of station columns
# station_columns_df = dask_df['path'].str.split('|', expand=True, n=max_stations)
# 
# # Step 3: Concatenate with relevant identifiers and location details
# train_rides_with_stations_df = dd.concat(
#     [dask_df[['ID', 'eva_nr', 'station', 'city']], station_columns_df],
#     axis=1
# )
# 
# # Step 4: Reshape to long format
# stations_long_format_df = train_rides_with_stations_df.melt(
#     id_vars=['ID', 'eva_nr', 'station', 'city'], 
#     var_name='station_sequence', 
#     value_name='station_name'
# ).dropna(subset=['station_name'])
# 
# # Compute the final DataFrame
# result_df = stations_long_format_df.compute()

In [12]:
# # Step 1: Split the 'path' column into separate station columns for each stop
# station_columns_df = train_rides_df['path'].str.split('|', expand=True)
# 
# # Step 2: Concatenate with relevant identifiers and location details to keep track of each train ride
# train_rides_with_stations_df = pd.concat([train_rides_df[['ID', 'eva_nr', 'station', 'city']], station_columns_df], axis=1)
# 
# # Step 3: Reshape to a long format with each station as a separate row, preserving ride details
# stations_long_format_df = train_rides_with_stations_df.melt(
#     id_vars=['ID', 'eva_nr', 'station', 'city'], 
#     var_name='station_sequence', 
#     value_name='station_name'
# )
# stations_long_format_df = stations_long_format_df.dropna(subset=['station_name'])

In [13]:
# # Step 4: Merge with 'station_mapping_df' to retrieve 'eva_nr' for each station
# stations_with_eva_df = stations_long_format_df.merge(
#     station_mapping_df[['station', 'eva_nr']], 
#     left_on='station_name',
#     right_on='station',
#     how='left',
#     suffixes=('', '_mapped')
# )
# 
# # Step 5: Drop duplicate or unnecessary columns for a clean DataFrame
# final_station_mapping_df = stations_with_eva_df.drop(columns=['station'])
# 
# # Step 6: Reset index for a cleaner, ordered display
# final_station_mapping_df = final_station_mapping_df.reset_index(drop=True)
# 
# # Display the final DataFrame
# final_station_mapping_df

In [14]:
import plotly.express as px

In [15]:
# # Generate a blank point graph
# fig_blank = px.scatter(train_rides_df, x="long", y="lat", title="Blank Point Graph of Longitude and Latitude")
# 
# # Show plot
# fig_blank.show()

In [16]:
# # Generate a map using longitude and latitude
# fig_map = px.scatter_mapbox(train_rides_df, lat="lat", lon="long", 
#                             mapbox_style="open-street-map", 
#                             title="Map of Train Stations Based on Longitude and Latitude")
# # Show plot
# fig_map.show()